In [17]:
import pandas as pd
import os
import time
from datetime import datetime

# Our goal here is to determine investing strategy based on
# fundamental analysis of stocks.
# features = various fundamental analysis variables
# class = (to keep it simple) Outperform / Underperform

# we can get pricing data from SEC or YahooFinance
# Here the data was obtained from YahooFinance into a zip file
# https://pythonprogramming.net/static/downloads/machine-learning-data/intraQuarter.zip
# Unzip this file in ../00-data

# For comparison, we will use S&P500 index against individual stocks
# Head on over to quandl.com, search for S&P 500, choose index, 
# Choose data from the year 2000 on-wards. After that, click on "download," 
# choosing CSV. Get file YAHOO-INDEX_GSPC.csv

# The reason for the Try and Except here is because some of our stock data may have been 
# pulled on a weekend day. If we hunt for a weekend day's value of the S&P 500, that date 
# just simply wont exist in the dataset, since there is only data for Monday-Friday.

path = "../00-data/intraQuarter"

def Key_Stats(gather="Total Debt/Equity (mrq)"):
    statspath = path+'/_KeyStats'
    stock_list = [x[0] for x in os.walk(statspath)]
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'DE Ratio',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change'])
    sp500_df = pd.DataFrame.from_csv("../00-data/YAHOO-INDEX_GSPC.csv")
    for each_dir in stock_list[1:25]:
        each_file = os.listdir(each_dir)
        ticker = each_dir.split("/")[-1]
        if len(each_file) > 0:
            for file in each_file:
                date_stamp = datetime.strptime(file, '%Y%m%d%H%M%S.html')
                unix_time = time.mktime(date_stamp.timetuple())
                full_file_path = each_dir+'/'+file
                source = open(full_file_path,'r').read()
                try:
                    value = float(source.split(gather+':</td><td class="yfnc_tabledata1">')[1].split('</td>')[0])

                    try:
                        sp500_date = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_date)]
                        sp500_value = float(row["Adjusted Close"])
                    except:
                        sp500_date = datetime.fromtimestamp(unix_time-259200).strftime('%Y-%m-%d')
                        row = sp500_df[(sp500_df.index == sp500_date)]
                        sp500_value = float(row["Adjusted Close"])


                    stock_price = float(source.split('</small><big><b>')[1].split('</b></big>')[0])
                        
                    

                    df = df.append({'Date':date_stamp,
                                    'Unix':unix_time,
                                    'Ticker':ticker,
                                    'DE Ratio':value,
                                    'Price':stock_price,
                                    'SP500':sp500_value}, ignore_index = True)
                except Exception as e:
                    pass

    save = gather.replace(' ','').replace(')','').replace('(','').replace('/','')+('.csv')
    print(save)
    df.to_csv('../00-data/'+save)
    
Key_Stats()

TotalDebtEquitymrq.csv
